# Proeycto de IA - ChatBox

Departamento de Ciencias de la Computación, Universidad de Chile.

CC6409: Taller de Desarrollo de Poryectod e IA - Otoño 2023

**Integrantes:**
- Garrido Martín
- Gómez Nahuel
- Santelices Gustavo


## Import

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report

from sklearn.base import BaseEstimator, TransformerMixin
import re
import numpy as np
import nlpaug.augmenter.word as naw # To synonym, see  to more info https://github.com/makcedward/nlpaug
import nlpaug.augmenter.char as nac
from mtranslate import translate
import pickle
from gensim.models.keyedvectors import KeyedVectors
import os
from nltk.tokenize import TweetTokenizer

## data

In [ ]:
# handmade
hand_nawel_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_nawel.csv", sep = ';')
hand_nawel_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_nawel.csv", sep = ';')
hand_nawel_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_nawel.csv", sep = ';')
hand_nawel_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_nawel.csv", sep = ';')

hand_tavo_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_tavo.csv", sep = ';')
hand_tavo_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_tavo.csv", sep = ';')
hand_tavo_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_tavo.csv", sep = ';')
hand_tavo_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_tavo.csv", sep = ';')

hand_martin_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_martin.csv", sep = ';', encoding = 'latin-1')
hand_martin_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_martin.csv", sep = ';')
hand_martin_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_martin.csv", sep = ';')
hand_martin_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_martin.csv", sep = ';')

In [ ]:
df_ = pd.concat([hand_nawel_0, hand_nawel_1, hand_nawel_2, hand_nawel_3,
                 hand_tavo_0, hand_tavo_1, hand_tavo_2, hand_tavo_3,
                 hand_martin_0, hand_martin_1, hand_martin_2, hand_martin_3])

df_.count()

class    345
text     345
dtype: int64

In [ ]:
df_['class'].value_counts()

0    101
1     89
2     87
3     68
Name: class, dtype: int64

## Data Augmentation

### ChatGPT sentences

In [ ]:
# Nahuel
gpt_nawel_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/asistire_nawel.csv", sep = ';')
gpt_nawel_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/no_asistire_nawel.csv", sep = ';')
gpt_nawel_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/reagendar_nawel.csv", sep = ';')
gpt_nawel_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/pedir_nawel.csv", sep = ';')

# Martin
gpt_martin_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/asistire_martin.csv", sep = ';')
gpt_martin_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/no_asistire_martin.csv", sep = ';')
gpt_martin_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/reagendar_martin.csv", sep = ';')
gpt_martin_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/pedir_martin.csv", sep = ';')

# Gustavo
#gpt_tavo_0 = pd.read_csv("", sep = ';')
#gpt_tavo_1 = pd.read_csv("", sep = ';')
#gpt_tavo_2 = pd.read_csv("", sep = ';')
#gpt_tavo_3 = pd.read_csv("", sep = ';')

In [ ]:
# Concat the chatgpt sentences
df_ = pd.concat([df_,
                 gpt_nawel_0, gpt_nawel_1, gpt_nawel_2, gpt_nawel_3,
                 gpt_martin_0, gpt_martin_1, gpt_martin_2, gpt_martin_3])

In [ ]:
df_['class'].value_counts()

0    185
1    178
2    162
3    141
Name: class, dtype: int64

### Translate augmentation (functions)

In [ ]:
# Extra : Translate Ideas
def sp_to_en(data):
    augmented_data = []
    for sentence in data:
        # Translate Spanish sentence to English
        translated_sentence = translate(sentence, "en")
        augmented_data.append(translated_sentence)

    return augmented_data

def en_to_sp(data):
    augmented_data = []
    for sentence in data:
        # Translate English sentence to Spanish
        translated_sentence = translate(sentence, "es")

        augmented_data.append(translated_sentence)

    return augmented_data

In [ ]:
# Import model
aug_fb = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de',
    to_model_name='facebook/wmt19-de-en'
)

In [ ]:
def Translate_aug(df_):
    # Getting the values with two or more words.
    df_english_aug = df_[df_['text'].apply(lambda x: len(x.split()) > 2)]

    # Traslate to English
    l_english_aug = sp_to_en(df_english_aug.text.tolist())

    # Find Synonyms
    english_syn_aug = aug_fb.augment(l_english_aug)

    # Translate to spanish
    l_english_aug = en_to_sp(df_english_aug.text.tolist())

    # Append text with label
    df_english = pd.DataFrame()
    df_english['class'] = df_english_aug['class']
    df_english['text'] = l_english_aug

    return df_english

### Data aug in spanish (functions)

In [ ]:
# Wordnet - spa
aug_wn = naw.SynonymAug(aug_src='wordnet', lang='spa')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
def spanish_aug(df_):
    # Getting the values with ten or more words.
    df_spanish_aug = df_[df_['text'].apply(lambda x: len(x.split()) > 10)]
    l_augmented = aug_wn.augment(df_spanish_aug.text.tolist())

    # Append text with label
    df_spanish = pd.DataFrame()
    df_spanish['class'] = df_spanish_aug['class']
    df_spanish['text'] = l_augmented

    return df_spanish

## Custom Features

In [ ]:
class PreProccesingTransformer(BaseEstimator, TransformerMixin):
    def preprocess(self,sentence):
      # Deleting all except: exclamation/question signs and accents
      new_word = re.sub(r"[^a-zA-ZáéíóúÁÉÍÓÚñÑ¡!¿?\s]", '', sentence)
      # Deleting double blank spaces
      new_sentence = new_word.replace('  ',' ').replace('\n','').strip()
      return new_sentence

    def transform(self, X, y=None):
        values = []
        for tweet in X:
            values.append(self.preprocess(tweet))

        return(np.array(values))

    def fit(self, X, y=None):
        return self

In [ ]:
class Doc2VecTransformer_auxiliar(BaseEstimator, TransformerMixin):
    def __init__(self, model, tokenizer,aggregation_func):
        self.model = model
        self.tokenizer=tokenizer
        # agregation func (np.min, np.max, np.mean, np.sum, ...)
        self.aggregation_func = aggregation_func

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        doc_embeddings = []
        for doc in X:
            # tokenization the doc
            tokens = self.tokenizer.tokenize(doc)
            selected_wv = []
            for token in tokens:
                if token in self.model.index_to_key:
                    selected_wv.append(self.model[token])

            if len(selected_wv) > 0:
                doc_embedding = self.aggregation_func(np.array(selected_wv), axis=0)
                doc_embeddings.append(doc_embedding)
            else:
                doc_embeddings.append(np.zeros(self.model.vector_size))

        return np.array(doc_embeddings)

In [ ]:
#current_dir = os.getcwd()
#wordvectors_file_vec = os.path.join(current_dir, "WordEmbeddings", "Spanis_Unannotated_Corpora.vec")

#cantidad = 600000
#wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

## Pipeline

### Run Pipeline

In [ ]:
# Defining threshold
thresholds = {
    0: 0.90,  # class 0
    1: 0.80,  # class 1
    2: 0.80,   # class 2
    3: 0.80   #class 3
}


def custom_predict(model, X, class_thresholds):
    # Get probabilities
    probabilities = model.predict_proba(X)

    # Aplied threshold
    modified_predictions = np.argmax(probabilities, axis=1)
    for class_label, threshold in class_thresholds.items():
        modified_predictions[probabilities[:, class_label] > threshold] = class_label

    return modified_predictions

In [ ]:
def run(dataset, pipeline):

    X_train, X_test, y_train, y_test = train_test_split(
        dataset['text'],
        dataset['class'],
        shuffle=True,
        test_size=0.2,
        random_state=42,
        stratify=dataset['class']
    )

    print(f"# Len Training Data: {len(X_train)}")
    print(f"# Len Testing Data: {len(X_test)}")

    print('Aplying data aug')

    df_to_aug = pd.DataFrame({'text': X_train, 'class': y_train})

    english_df = Translate_aug(df_to_aug)
    spanish_df = spanish_aug(df_to_aug)

    X_train = pd.concat([X_train, english_df['text'], spanish_df['text']])
    y_train = pd.concat([y_train, english_df['class'], spanish_df['class']])

    print(f"# NEW Len Training Data: {len(X_train)}")
    print(f"# NEW Len Testing Data: {len(X_test)}")

    pipeline.fit(X_train, y_train)


    # Classes propbailities
    probabilities = pipeline.predict_proba(X_test)


    # Predicted labels
    predicted_labels = pipeline.predict(X_test)
    #predicted_labels = custom_predict(pipeline, X_test, thresholds)


    print(classification_report(y_test, predicted_labels))


    return pipeline, X_train, X_test, y_train, y_test, predicted_labels, probabilities


## Defining Pipelines

In [ ]:
# simple Pipeline only BOW

def get_experiment_0_pipeline():

    return Pipeline(
        [
            (
                "features",
                FeatureUnion(
                    [
                        ("bow", CountVectorizer()),
                    ]
                ),
            ),
            ("clf", MultinomialNB()),
        ]
    )

In [ ]:
# simple Pipeline only Preprocessing + BOW

def get_experiment_1_pipeline():

    return Pipeline(
        [
            ("preprocessing", PreProccesingTransformer()),
            (
                "features",
                FeatureUnion(
                    [
                        ("bow", CountVectorizer())
                    ]
                ),
            ),
            ("clf", MultinomialNB()),
        ]
    )

In [ ]:
# simple Pipeline only BOW

def get_experiment_2_pipeline():

    return Pipeline(
        [
            (
                "features",
                FeatureUnion(
                    [
                        ("bow", CountVectorizer()),
                        ('d2v', Doc2VecTransformer_auxiliar(wordvectors,TweetTokenizer(),np.sum)),
                    ]
                ),
            ),
           ("clf", GradientBoostingClassifier(n_estimators=200, learning_rate=0.7, max_depth=1, random_state=0)),
        ]
    )

In [ ]:
# simple Pipeline only BOW

def get_experiment_3_pipeline():

    return Pipeline(
        [
            (
                "features",
                FeatureUnion(
                    [
                        ("bow", CountVectorizer()),
                        ('d2v', Doc2VecTransformer_auxiliar(wordvectors,TweetTokenizer(),np.sum)),
                    ]
                ),
            ),
           ('clf', SVC(probability=True)),
        ]
    )

## Implementación del modelo

In [ ]:
# Simple Pipeline Test
pipeline0 = get_experiment_0_pipeline()

prob = run(df_, pipeline0)

# Len Training Data: 532
# Len Testing Data: 134
Aplying data aug
# NEW Len Training Data: 1181
# NEW Len Testing Data: 134
              precision    recall  f1-score   support

           0       0.94      0.78      0.85        37
           1       0.84      0.86      0.85        36
           2       0.82      0.94      0.87        33
           3       0.93      0.93      0.93        28

    accuracy                           0.87       134
   macro avg       0.88      0.88      0.88       134
weighted avg       0.88      0.87      0.87       134



In [ ]:
# Save the model
with open('model0.bin', 'wb') as fil:
    pickle.dump(pipeline0, fil)

In [ ]:
# Simple Pipeline Test (+ preprocessing)
pipeline1 = get_experiment_1_pipeline()

_ = run(df_, pipeline1)

# Len Training Data: 532
# Len Testing Data: 134
Aplying data aug
# NEW Len Training Data: 1181
# NEW Len Testing Data: 134
              precision    recall  f1-score   support

           0       0.94      0.78      0.85        37
           1       0.83      0.83      0.83        36
           2       0.79      0.94      0.86        33
           3       0.93      0.93      0.93        28

    accuracy                           0.87       134
   macro avg       0.87      0.87      0.87       134
weighted avg       0.87      0.87      0.87       134



In [ ]:
# Save the model
with open('model1.bin', 'wb') as fil:
    pickle.dump(pipeline1, fil)

In [ ]:
# Word 2 vec text
from sklearn.svm import SVC
pipeline2 = get_experiment_2_pipeline()

prob = run(df_, pipeline2)

# Len Training Data: 532
# Len Testing Data: 134
Aplying data aug
# NEW Len Training Data: 1181
# NEW Len Testing Data: 134
              precision    recall  f1-score   support

           0       0.87      0.92      0.89        37
           1       0.91      0.89      0.90        36
           2       0.87      0.79      0.83        33
           3       0.90      0.96      0.93        28

    accuracy                           0.89       134
   macro avg       0.89      0.89      0.89       134
weighted avg       0.89      0.89      0.89       134



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
pipeline3 = get_experiment_3_pipeline()

prob = run(df_, pipeline3)

# Len Training Data: 532
# Len Testing Data: 134
Aplying data aug
# NEW Len Training Data: 1181
# NEW Len Testing Data: 134
              precision    recall  f1-score   support

           0       0.76      0.86      0.81        37
           1       0.89      0.69      0.78        36
           2       0.83      0.91      0.87        33
           3       0.93      0.93      0.93        28

    accuracy                           0.84       134
   macro avg       0.85      0.85      0.85       134
weighted avg       0.85      0.84      0.84       134



In [ ]:
## User response

In [ ]:
map_label= {
    0: 'Asistir a la',
    1: 'Cancelar la',
    2: 'Reagendar la',
    3: 'Pedir una'
}

options = {
    0: 'Su reserva a quedado confirmada para el DD de MM',
    1: 'Su reserva con fecha DD de MM a quedado cancelada',
    2: 'Las fechas para reagendar son las siguientes',
    3: 'Las fechas próximas son las siguientes'
}

def manage_mesages(target):
    # Make a prediction
    label = pipeline1.predict([target])[0]
    print(f"usted quiere {map_label[label]} hora")
    print(options[label])

In [ ]:
manage_mesages('Puedo cancelar la hora??')

usted quiere Cancelar la hora
Su reserva con fecha DD de MM a quedado cancelada


## Testing RNN


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
# Creation Of tuples (class, text)
dataset = tuple(
    (row['class'], row['text'].lower()) # With lower
    for index, row in df_.iterrows()
)

from random import sample
for example in sample(dataset, 5):
    print("Example:\n", example)

Example:
 (3, 'tiene hora para la licencia el proximo sabado?')
Example:
 (3, ' holas necesito una atencion para renovar la licencia')
Example:
 (2, 'me surgió una urgencia, necesito cambiar mi hora')
Example:
 (0, 'no me vai a fallar.')
Example:
 (0, 'si voy a ir pero puede indicar a que hora es?')


In [ ]:
import torch
from torch.utils.data.dataset import random_split
train_len = int(len(dataset) * .8)

train_split, validation_split = random_split(dataset, [train_len, len(dataset) - train_len])


In [ ]:
# To vocabulary
!python -m spacy download es

2023-07-04 23:19:47.397746: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'es' are deprecated. Please use the
full pipeline package name 'es_core_news_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 44.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("spacy", "es")
min_freq = 1
vocab = build_vocab_from_iterator((tokenizer(text[1]) for text in train_split), min_freq=min_freq)
labels = list({doc[0] for doc in train_split})
label_map = {label: index for index, label in enumerate(labels)}

UNK_IDX = 0
vocab.set_default_index(UNK_IDX)

vocab.insert_token('<pad>', 1)

stoi = vocab.get_stoi()

print("Vocabulary len:", len(vocab))
print("Random Words:", sample(vocab.get_itos(), 5))
print("Label's number:", len(labels))

Vocabulary len: 729
Random Words: ['agendeme', 'tal', 'podí', 'cuadra', 'rajarme']
Label's number: 4


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Red neuronal con una sola capa oculta
class ArgumentClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class, hidden_size, pad_idx):
        super().__init__()

        # capa de embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim, pad_idx)
        # self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, mode="mean")

        # capas de la MLP
        self.fc = nn.Linear(embed_dim, num_class)
        # self.fc1 = nn.Linear(embed_dim, hidden_size)
        # self.fc2 = nn.Linear(hidden_size, num_class)

    def forward(self, batch):
        # La representacion de un documento sera el promedio de los
        # embeddings de sus palabras.
        # (B, N, 1) -> (B, N, E)
        h = self.embedding(batch)
        # (B, N, E) -> (B, E)
        h = h.mean(dim=1)
        # h = self.embedding(batch)

        # computar las capas de la red MLP
        h = self.fc(h)
        # h = F.relu(self.fc1(h))
        # h = self.fc2(h)

        return h
        # return torch.softmax(h, -1)

In [ ]:
from itertools import zip_longest

# creamos lista de tensores
train_dataset, validation_dataset = [
    [
        (
            label_map[item[0]],
            torch.tensor([vocab[token] for token in tokenizer(item[1])]),
        ) for item in split
    ] for split in [train_split, validation_split]
]

In [ ]:
def generate_batch(batch):
    return (
        # En este caso como los labels son números,
        # el tensor es de una sola dimension de tamaño batch_size
        torch.tensor([item[0] for item in batch]),

        # En este caso se retorna un tensor de 2 dimensiones, batch_size x N,
        # donde N es mayor largo de los ejemplo en el batch. Aca se realiza
        # padding de los ejemplos mas cortos.
        torch.tensor(
            list(
                zip(
                    *zip_longest(
                        *[item[1] for item in batch], fillvalue=vocab["<pad>"]
                    )
                )
            )
        ),
    )

In [ ]:
# Ahora creamos funciones para entrenar y validar el modelo
from torch.utils.data import DataLoader


def train_func(train_dataset):

    # Entranamos el modelo
    train_loss = 0
    train_acc = 0
    data = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=generate_batch,
    )
    for i, (cls, text) in enumerate(data):
        optimizer.zero_grad()
        cls, text = cls.to(device), text.to(device)
        output = model(text)

        loss = criterion(output, cls)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        train_acc += (output.argmax(1) == cls).sum().item()

    # Ajustar el learning rate
    # scheduler.step()

    return train_loss / len(train_dataset), train_acc / len(train_dataset)


def test(test_dataset):
    test_loss = 0
    acc = 0
    data = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, collate_fn=generate_batch
    )
    for cls, text in data:
        cls, text = cls.to(device), text.to(device)
        with torch.no_grad():
            output = model(text)
            loss = criterion(output, cls)
            test_loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return test_loss / len(test_dataset), acc / len(test_dataset)

In [ ]:
import time

N_EPOCHS = 20
LEARN_RATE = 2.0
STEP_SIZE = 1
BATCH_SIZE = 10
EMBED_DIM = 100
HIDDEN_SIZE = 1024

device = "cuda" if torch.cuda.is_available() else "cpu"

model = ArgumentClassifier(
    vocab_size=len(vocab),
    embed_dim=EMBED_DIM,
    num_class=len(labels),
    hidden_size=HIDDEN_SIZE,
    pad_idx=vocab["<pad>"],
).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LEARN_RATE)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, STEP_SIZE)


for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(train_dataset)
    valid_loss, valid_acc = test(validation_dataset)

    secs = int(time.time() - start_time)
    mins = secs // 60
    secs = secs % 60

    print(
        f"Epoch: {epoch + 1}", f" | time in {mins} minutes, {secs} seconds",
    )
    print(
        f"\tLoss: {train_loss:.4f}(train)\t|"
        f"\tAcc: {train_acc * 100:.1f}%(train)"
    )
    print(
        f"\tLoss: {valid_loss:.4f}(valid)\t|"
        f"\tAcc: {valid_acc * 100:.1f}%(valid)"
    )

Epoch: 1  | time in 0 minutes, 0 seconds
	Loss: 0.0987(train)	|	Acc: 61.7%(train)
	Loss: 0.3572(valid)	|	Acc: 28.4%(valid)
Epoch: 2  | time in 0 minutes, 0 seconds
	Loss: 0.0684(train)	|	Acc: 75.8%(train)
	Loss: 0.1168(valid)	|	Acc: 66.4%(valid)
Epoch: 3  | time in 0 minutes, 0 seconds
	Loss: 0.0420(train)	|	Acc: 85.0%(train)
	Loss: 0.0530(valid)	|	Acc: 82.8%(valid)
Epoch: 4  | time in 0 minutes, 0 seconds
	Loss: 0.0291(train)	|	Acc: 90.8%(train)
	Loss: 0.0539(valid)	|	Acc: 84.3%(valid)
Epoch: 5  | time in 0 minutes, 0 seconds
	Loss: 0.0233(train)	|	Acc: 92.7%(train)
	Loss: 0.0589(valid)	|	Acc: 74.6%(valid)
Epoch: 6  | time in 0 minutes, 0 seconds
	Loss: 0.0183(train)	|	Acc: 94.9%(train)
	Loss: 0.1029(valid)	|	Acc: 64.2%(valid)
Epoch: 7  | time in 0 minutes, 0 seconds
	Loss: 0.0153(train)	|	Acc: 96.8%(train)
	Loss: 0.0669(valid)	|	Acc: 76.1%(valid)
Epoch: 8  | time in 0 minutes, 0 seconds
	Loss: 0.0114(train)	|	Acc: 98.7%(train)
	Loss: 0.0403(valid)	|	Acc: 85.8%(valid)
Epoch: 9  | time

In [ ]:
model.eval()


ArgumentClassifier(
  (embedding): Embedding(729, 100, padding_idx=1)
  (fc): Linear(in_features=100, out_features=4, bias=True)
)

In [ ]:
text = 'Es posible cambiar mi hora ahora??'
input_tensor = torch.tensor([vocab[token] for token in text], device=device).unsqueeze(0)
output = model(input_tensor)
predicted_class = torch.argmax(output, dim=1)
print(predicted_class)

tensor([2])
